In [1]:
%matplotlib inline
import psluncert

# Defining the System

Set up the system. UncertaintyCalc() can take a string representing the function, or a list of strings for multiple functions. Specify input variables using set_input(), providing the variable name (matching the variables in the function string), nominal value, and uncertainty parameters.

In [2]:
u = psluncert.UncertCalc('J*V')
u.set_input('J', nom=4, unc=.04, k=2)
u.set_input('V', nom=20, unc=.2, k=2)
u.calculate()




Method            |Mean  |Standard Uncertainty 
------------------|------|---------------------
GUM Approximation |80.00 |0.57                 
Monte Carlo       |80.00 |0.57                 



Alternatively, the inputs and uncertainties can be specified in a list/dictionary and passed in to the UncertaintyCalc().

In [3]:
inputs = [{'name': 'J', 'nom': 4,  'uncerts':[{'name': 'u(J)', 'unc':0.04, 'k':2}]},
          {'name': 'V', 'nom': 20, 'uncerts':[{'name': 'u(V)', 'unc':0.2,  'k':2}]}]
u = psluncert.UncertCalc('J*V', inputs=inputs)
u.calculate()




Method            |Mean  |Standard Uncertainty 
------------------|------|---------------------
GUM Approximation |80.00 |0.57                 
Monte Carlo       |80.00 |0.57                 



If a variable has multiple uncertainty components, each one can be entered using the set_uncert() method.

In [4]:
u = psluncert.UncertCalc('J*V')
u.set_input('J', nom=4, unc=.04, k=2)
u.set_input('V', nom=20)
u.set_uncert('V', name='u(typeA)', std=.1, k=2)
u.set_uncert('V', name='u(typeB)', std=.15, k=2)
u.calculate()




Method            |Mean  |Standard Uncertainty 
------------------|------|---------------------
GUM Approximation |80.00 |0.82                 
Monte Carlo       |80.00 |0.82                 



# Viewing Results

The calculate() function returns an output report object (uoutput.CalcOutput) which as reporting and plotting functions. The output report is also accessible using the 'out' member of the u object.

In [5]:
out = u.calculate()
out.report_summary()



Method            |Mean  |Std. Uncertainty |95% Coverage   |k     |Deg. Freedom 
------------------|------|-----------------|---------------|------|-------------
GUM Approximation |80.00 |0.82             |± 1.6          |1.960 |inf          
Monte Carlo       |80.00 |0.83             |[78.39, 81.62] |1.959 |-            

![][FIG0]




[FIG0]: 

In [6]:
out.report_expanded()

### GUM Approximation


Interval |Min   |Max   |k     |Deg. Freedom |Expanded Uncertainty 
---------|------|------|------|-------------|---------------------
99.00%   |77.9  |82.1  |2.576 |inf          |2.1                  
95.00%   |78.4  |81.6  |1.960 |inf          |1.6                  
90.00%   |78.6  |81.4  |1.645 |inf          |1.4                  
68.00%   |79.18 |80.82 |0.994 |inf          |0.82                 

### Monte Carlo

Symmetric Coverage Intervals

Interval |Min   |Max   |k     
---------|------|------|------
99.00%   |77.89 |82.14 |2.578 
95.00%   |78.39 |81.62 |1.959 
90.00%   |78.64 |81.36 |1.645 
68.00%   |79.18 |80.82 |0.994 



In [7]:
out.report_sens()


Variable |GUM Sensitivity |GUM Proportion |MC Sensitivity |MC Proportion 
---------|----------------|---------------|---------------|--------------
      |20              |23.53%         |20             |23.50%        
      |4               |76.47%         |4              |76.33%        



## Correlating Inputs

Correlate the inputs by using correlate_vars() to specify coefficients one pair at a time, or using set_correlation() to specify the full correlation matrix. 

In [8]:
u.correlate_vars('J', 'V', -.75)
u.calculate()




Method            |Mean  |Standard Uncertainty 
------------------|------|---------------------
GUM Approximation |80.00 |0.50                 
Monte Carlo       |80.00 |0.75                 



# Plotting outputs

Use the plot_* functions in the output object histograms and other plots. Note the semicolon suppreses duplicate figures due to the pyplot interface.

In [9]:
u.out.plot_pdf();

In [10]:
u.out.plot_xscatter(points=5000);

# Full report
The full report, as saved by the GUI, can be shown using u.out.report_all()

In [11]:
u.out.report_all()

## Summary



Method            |Mean  |Std. Uncertainty |95% Coverage   |k     |Deg. Freedom 
------------------|------|-----------------|---------------|------|-------------
GUM Approximation |80.00 |0.50             |± 0.97         |1.960 |inf          
Monte Carlo       |80.00 |0.75             |[78.52, 81.46] |1.961 |-            

![][FIG0]

## Standardized Input Values


Variable |Mean  |Std. Uncertainty |Deg. Freedom |Description 
---------|------|-----------------|-------------|------------
      |4.000 |0.020            |inf          |--          
      |20.00 |0.18             |inf          |--          

---

## Uncertainty Components


Variable |Component  |Description |Standard Uncertainty |Deg. Freedom 
---------|-----------|------------|---------------------|-------------
      |     |--          |0.020                |inf          
      |           |            |0.18                 |inf          
         | |--          |0.10                 |inf          
         | |--          |0.15                 |inf          

---

## Sensitivity Coefficients


Variable     |GUM Sensitivity |GUM Proportion |MC Sensitivity |MC Proportion 
-------------|----------------|---------------|---------------|--------------
          |20              |64.69%         |20             |28.64%        
          |4               |210.24%        |5.54           |178.65%       
Correlations |                |-174.93%       |               |              

---

## Expanded Uncertainties

### GUM Approximation


Interval |Min   |Max   |k     |Deg. Freedom |Expanded Uncertainty 
---------|------|------|------|-------------|---------------------
99.00%   |78.7  |81.3  |2.576 |inf          |1.3                  
95.00%   |79.03 |80.97 |1.960 |inf          |0.97                 
90.00%   |79.18 |80.82 |1.645 |inf          |0.82                 
68.00%   |79.51 |80.49 |0.994 |inf          |0.49                 

### Monte Carlo

Symmetric Coverage Intervals

Interval |Min   |Max   |k     
---------|------|------|------
99.00%   |78.06 |81.91 |2.573 
95.00%   |78.52 |81.46 |1.961 
90.00%   |78.76 |81.22 |1.647 
68.00%   |79.25 |80.74 |0.995 

## GUM Derivation

### Function:



GUM formula for combined uncertainty:



### Input Definitions:


Variable |Std. Uncertainty 
---------|-----------------
      |          
      |          

Correlation coefficients:


### Sensitivity Coefficients:





### Simplified combined uncertainty:



### Effective degrees of freedom:



---

## GUM Validity

### Comparison to Monte Carlo 95.00% Coverage

2 significant digits. δ = 0.005.


95.00% Coverage   |Lower Limit                   |Upper Limit                   
------------------|------------------------------|------------------------------
GUM               |79.03                         |80.97                         
MC                |78.53                         |81.46                         
abs(GUM - MC)     |0.49                          |0.49                          
abs(GUM - MC) < δ |<font color="red">FAIL</font> |<font color="red">FAIL</font> 

---

## Monte Carlo Inputs

![][FIG1]

## Monte Carlo Convergence

![][FIG2]




[FIG0]: 
[FIG1]: 
[FIG2]: 

# Setup Files

The calculator parameters can be loaded from a saved confiuration file (yaml format).

In [12]:
u = psluncert.UncertCalc.from_configfile('../ex_magphase.yaml')
u.calculate()




Method            |Mean   |Standard Uncertainty 
------------------|-------|---------------------
GUM Approximation |0.0010 |0.010                
Monte Carlo       |0.0126 |0.0066               




Method            |Mean   |Standard Uncertainty 
------------------|-------|---------------------
GUM Approximation |0.0    |10                   
Monte Carlo       |0.0031 |1.7                  

